### PCP Assignment 2 Module 2 - similarity_module

In [1]:
from load_dataset_module import read_File
from load_dataset_module import Artist, Song, Track, IterRegistry, Extras   # Classes
import pandas as pd
import numpy as np
import math
from scipy.spatial import distance 
from numpy.linalg import norm
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity

# Disable warnings, these are not required for the user to see (might be removed)
import warnings
warnings.filterwarnings('ignore')

# These will change over time
from elasticsearch import Elasticsearch, RequestsHttpConnection
from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

In [2]:
# Creating logging capabilities

import logging
logging.warning('Watch out!')  # will print a message to the console
logging.info('I told you so')  # will not print anything

In [3]:
#print(help(Track))

In [4]:
# new_artist = read_wholeFile()
# #print(new_artist[45].getName())

# fname = 'Carl'
# lname = "W"

# i = 0
# for i in range(0, len(new_artist)):
#     if fname + " " + lname in new_artist[i].getName():
#         print("Index", i ,":",new_artist[i].getName())

In [5]:
def search_artist(list_name):
    try:
        fName = str(input("Please enter the first name of the artist you want to find: ").strip().capitalize())
        lName = str(input("Please enter the surname of the artist you want to find. At least an Initial entry is required: "\
                         ).strip().capitalize())
    
        # Create empty dictionary and lists for the inputs
        results_dict = {}
        result_names = []
        result_songs = []
        result_id = []
        num = 0 # Index for printing results
        
        # Loop through the dictionary, extract all matching artist + song values
        for i in range(0, len(list_name)): # range of the whole class-based list
            if fName + " " + lName in list_name[i].getName():
                result_names.append(list_name[i].getName())
                result_songs.append(list_name[i].getSongName())
                result_id.append(i) 
                
        if len(result_id) == 0: # If the length of the list above is 0, we found no matches
            print("Your search returned no results.")
        else:
            # return an assignable dictionary using the First Name and Surname Initial for the indexable ID
            print("Your search returned {} results.".format(len(result_id)))
            output = str(input("Would you like to view results? ").strip().capitalize())
            if output == "Yes":
                for k in range(0, len(result_id)):
                    print("ID: {} | {} | {} | ".format(result_id[num], result_names[num], result_songs[num]))
                    num+=1
            else:
                print("Search complete.")
    
    # Error handling for the function is written here
    except KeyError:
        return(print("You have entered an incorrect value, please check your entry."))
    except TypeError:
        return(print("You can't enter a number or symbol here, please enter a string dictionary name."))
    except IndexError:
        return(print("You must enter at least an Initial into the Surname box."))

In [6]:
def search_song(list_name):
    try:
        song_name = input("Please insert the word/s you would like to find in a song: ").capitalize()
        song_name = song_name.rstrip()   # remove end of input whitespace
        song_name = song_name.split(' ') # split the words into individual words to detect how many we have

        result_id = []
        result_names = []
        result_songs = []
        num = 0 # Index for printing results
        
        if len(song_name) == 1: # Check if the input is 1 word or many words
            song_name = ''.join(song_name) # if it is 1 word, reconnect the word through the join command
            for i in range(0, len(list_name)): # range of the whole class-based list
                if song_name in list_name[i].getSongName():
                    # Add matching values to assignable lists 
                    result_id.append(i)      
                    result_names.append(list_name[i].getName())
                    result_songs.append(list_name[i].getSongName()) 
                    
        else: # Loop the list and match each word with a value
            for i in range(0, len(list_name)): # range of the whole dictionary +1 as end of dictionary is missed otherwise
                song_name = [item.capitalize() for item in song_name] # capitalise each string
                for j in range(0, len(song_name)):
                    if song_name[j] in list_name[i].getSongName(): # search each word for a match
                        # Add matching values to assignable lists
                        result_id.append(i)     
                        result_names.append(list_name[i].getName())
                        result_songs.append(list_name[i].getSongName())

        if len(result_id) == 0: # If the length of the list is 0, we found no matches
            print("Your search returned no results.")
        else:
            print("Your search returned {} results.".format(len(result_id)))
            output = str(input("Would you like to view results? ").capitalize().rstrip())
            if output == "Yes":
                for k in range(0, len(result_id)):
                    print("ID: {} | {} | {}".format(result_id[num], result_names[num], result_songs[num]))
                    num+=1
            else:
                print("Search complete.")
                    
    # Error handling for the function is written here
    except KeyError as keyerror:
        return(print("You have entered an incorrect value, please check your entry.", keyerror))
    except TypeError as typeerror:
        return(print("You can't enter a number or symbol here, please enter a string dictionary name.", typeerror)) 

In [7]:
song_searcher = read_File()

Finished reading the command for file loading.


In [10]:
song_searcher[0].getExtras()

'Duration: 158648, Explicit: 0, Instrumentalness: 0.563, Key: 10, Mode: 1, Release Date: 1928'

In [8]:
# search_for = search_artist(song_searcher)
# search_for

In [11]:
class similarity_metric(object):
    def __init__(self, value1, value2):
        __metaclass__ = IterRegistry
        _registry = []
        
    # These do work for x by x conversion however
        # Our defined target chosen by the user
        self.target = value1
        # Our defined library of all other values
        self.library = value2
        
    # Returns object in string format for readability
    def __repr__(self):
        return f'Target: {self.target}, Comparison: {self.library}'
    #TODO change the above names later
    
    def euclidean(self):
        a = np.array(self.target)
        b = np.array(self.library)
        return np.linalg.norm(a - b)
        
    def manhattan(self):
        a = np.array(self.target)
        b = np.array(self.library)
        #return np.abs(a - b) # This works
        return manhattan_distances([a, b])
        
    def cosine(self):
        a = np.array(self.target)
        b = np.array(self.library)
        #return np.dot(a, b)/(norm(a)*norm(b))
        #return cosine_similarity([[a], [b]])
        return 1 - distance.cosine(a, b)
        #return distance.cosine(a, b)

    def jaccard(self):
        a = np.array(self.target)
        b = np.array(self.library)
        return distance.jaccard(a, b)
    
    def pearson(self):
        a = np.array(self.target)
        b = np.array(self.library)
        return np.corrcoef([a, b])


In [12]:
# Class for Similar Artist, inherits from the SM class
class similar_artist(similarity_metric):
    pass

In [13]:
# Class for Similar Music, inherits from the SM class
class similar_music(similarity_metric):
    pass

In [14]:
# Class for All similarities, inherits from the SM class
class similar_all(similarity_metric):
    pass

In [15]:
# Function to create the df and do the similarity comparison

def similarity_caller(list_name):
    pass

In [37]:
# Create a dataframe from the class object Track

from sklearn import preprocessing

df = pd.DataFrame.from_records([s.to_dict() for s in song_searcher])
df = df.drop(['Music_ID', 'Release Date', 'Duration in ms'], axis = 1)
# TODO these need to be removed from the Track class' to_dict method

# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

#print(df['Acousticness'][4000])

In [38]:
df.head()

,Acousticness,Danceability,Energy,Liveness,Loudness,Popularity,Speechiness,Tempo,Valence,Explicit,Key,Instrumentalness
0,0.995,0.708,0.1950,0.1510,-12.428,0,0.0506,118.469,0.7790,0,10,0.563
1,0.994,0.379,0.0135,0.0763,-28.454,0,0.0462,83.972,0.0767,0,8,0.901
2,0.604,0.749,0.2200,0.1190,-19.924,0,0.9290,107.177,0.8800,0,5,0.000
3,0.995,0.781,0.1300,0.1110,-14.734,0,0.0926,108.003,0.7200,0,1,0.887
4,0.990,0.210,0.2040,0.0980,-16.829,1,0.0424,62.149,0.0693,0,11,0.908


https://towardsdatascience.com/how-to-build-a-recommendation-engine-quick-and-simple-aec8c71a823e
https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html
https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-significantterms-aggregation.html
https://realpython.com/build-recommendation-engine-collaborative-filtering/

In [39]:
# Testing out 9 feature values vs 9 feature values using the metrics

feature = 'Tempo'
id_num = 30
feature_list = []
manhattan_list = []
i = 0

#one = np.array([[20,30,40,20,10,30,20,10,23.44]])
#two = np.array([[10,1000,328.4,100,109,293,10,45,3291]])

#print(df[1])

one = np.array(df)[id_num].round(3)
two = np.array(df)[90000].round(3)
#one = np.sum(one)
#two = np.sum(two)

new_entry = similarity_metric(one,two) 
print(new_entry.pearson())
print(new_entry.cosine())
print(new_entry.manhattan())
print(new_entry.euclidean())
print(new_entry.jaccard())
print('\n')
print(one)
print(two)

one = np.sum(one)
two = np.sum(two)

# for val in df[id_num]:
#     print(val)

[[1.         0.99597011]
 [0.99597011 1.        ]]
0.996276401017956
[[ 0.    53.127]
 [53.127  0.   ]]
46.14960778814919
1.0


[  0.995   0.767   0.23    0.133 -10.728   0.      0.199 119.985   0.868
   0.      2.      0.605]
[  0.984   0.361   0.295   0.72  -11.717   1.      0.078  73.966   0.18
   0.      5.      0.846]


In [29]:
len(df)

169909

In [31]:
%%time

cosine_list = []
id_num = 300
i = 0

target = np.array(df)[id_num].round(2)

for i in range(0, 15):
    compare = np.array(df)[i].round(2)
    result = (similarity_metric(target, compare)).euclidean()
    cosine_list.append(result)
    #i += 1 
print(cosine_list)

[9.033985831292851, 44.832061072406646, 20.23000741472923, 18.949651711838925, 64.78488095227156, 63.49951574618502, 46.44036606229542, 6.801683615105891, 9.152928493110823, 45.89545838097709, 15.795385402072332, 43.43815718927312, 30.736671908324748, 51.96411261630472, 46.71674004037524]
Wall time: 122 ms


## Testing Manhattan - need to fix, dont run for now

In [77]:
# The code basis for one target to all values in the same column


feature = 'Tempo'
feature_list = []
manhattan_list = []
i = 0

for val in df[feature]: 
    feature_list.append(similarity_metric(df[feature][45000], val))
    manhattan_score = feature_list[i].manhattan()
    manhattan_list.append(manhattan_score)
    i += 1 

#print(euclidean_list)
#print(tempo_list) #Indent this if you want one-by-one printing
#tempo_list = np.array(tempo_list)

### Need this for later

In [ ]:
# for i in range(0,9): # Loop through the 9 expected features
# for value in feature_list, feature_list2: # take values over the loop and compare them
# x = (feature_list[i]) 
# y = (feature_list2[i]) 
# distance = math.sqrt((x - y) ** 2 ) #one-dimensional euclidean formula
# print(key_list[i].strip('[]').strip(' '), round(distance, 3)) # print all feature metrics

In [ ]:
print(len(manhattan_list))
print(manhattan_list[25:35])

In [ ]:
for i in range(0, len(manhattan_list)):
    if abs(manhattan_list[i]) <= 0:
        print(i,':', manhattan_list[i])

## Testing Cosine

In [79]:
%%time
# The code basis for one target to all values in the same column
 
feature = 'Valence'
feature_list = []
cosine_list = []
i = 0

for val in df[feature]:   
    feature_list.append(similarity_metric(df[feature][45000], val))
    cosine_score = feature_list[i].cosine()
    cosine_list.append(cosine_score)
    i += 1

#print(euclidean_list)
#print(tempo_list) #Indent this if you want one-by-one printing
#tempo_list = np.array(tempo_list)

Wall time: 7.91 s


In [82]:
print(len(cosine_list))
print(cosine_list[25:100])

169909
[53.473520549894594, 44.2151749968266, 7.628912111172867, 46.99028197404225, 14.362332679617197, 8.48883384217172, 26.155756536563793, 17.20782961328941, 46.25187347556853, 6.511581989040758, 6.002157945272691, 10.447645667804775, 17.173170936085157, 42.32441493984293, 13.770442984886161, 46.60631609556798, 68.06466337241373, 55.43081453487762, 14.366962100597323, 14.61406514286836, 43.67122279029979, 12.724484272456774, 43.25757274743926, 7.9005126415948475, 60.18996760258306, 52.894231254457225, 68.62242344889897, 48.723443022840655, 15.757163450316805, 62.889174744148136, 20.86840195127552, 50.495082928934764, 55.73413496233703, 61.48053513104778, 12.567481848007574, 12.475676334371618, 5.611318918044131, 3.8666005741477845, 16.388319010807667, 6.339723968754472, 12.890674924145744, 48.574239057343966, 51.598333306416, 10.722210592970093, 22.133072538624177, 67.82277567307312, 12.929945862222313, 9.103944200180486, 6.604271345122036, 40.29452320105054, 23.681211962228616, 89.

In [85]:
for i in range(0, len(cosine_list)):
    if cosine_list[i] <= 1:
        print(i,':', cosine_list[i])

300 : 0.0
1028 : 0.9799999999999985
8916 : 0.9388290579226874
17264 : 0.3784177585684866
24238 : 0.8413085046521322
33615 : 0.6416385275215353
40773 : 0.5678908345800245
48120 : 0.7008566187174121
49062 : 0.835224520712848
49174 : 0.6394528911499308
56535 : 0.6407027391856482
56621 : 0.7092249290598885
63245 : 0.7826237921249282
63971 : 0.8301807032206885
70743 : 0.5187484939737181
78176 : 0.896827742657421
78707 : 0.5872818744010422
88154 : 0.518459255872636
88170 : 0.6987846592477529
88281 : 0.6113100686231161
89227 : 0.41291645644125224
97873 : 0.49568134925574875
98198 : 0.660302960768763
98408 : 0.9098901032542328
98479 : 0.9428679653058576
98677 : 0.9362691920596327
99658 : 0.5479963503528081
107351 : 0.6542935121182228
108458 : 0.3096772513440363
117593 : 0.6000833275470986
117911 : 0.5452522352086228
118784 : 0.7619711280619467
126443 : 0.5989156868875617
126604 : 0.9996999549864966
126818 : 0.7188184749990828
126963 : 0.6624952830020752
127335 : 0.9062008607367339
127610 : 0.8

## Testing Jaccard

In [92]:
%%time

feature = 'Valence'
feature_list = []
jaccard_list = []
id_num = 45000
i = 0

for val in df[feature]:   
    
    feature_list.append(similarity_metric(df[feature][id_num], val))
    jaccard_score = feature_list[i].jaccard()
    jaccard_list.append(jaccard_score)
    i += 1

Wall time: 5.34 s


In [93]:
print(len(jaccard_list))
print(jaccard_list[25:35])

169909
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [94]:
for i in range(0, len(jaccard_list)):
    if jaccard_list[i] <= 0:
        print(i,':', jaccard_list[i])

1245 : 0.0
1340 : 0.0
5276 : 0.0
6050 : 0.0
6182 : 0.0
6233 : 0.0
7058 : 0.0
7555 : 0.0
8361 : 0.0
8943 : 0.0
9386 : 0.0
9555 : 0.0
9749 : 0.0
11551 : 0.0
12517 : 0.0
12818 : 0.0
15231 : 0.0
16276 : 0.0
18650 : 0.0
20355 : 0.0
23191 : 0.0
23597 : 0.0
23709 : 0.0
24481 : 0.0
24562 : 0.0
24676 : 0.0
26056 : 0.0
26609 : 0.0
26660 : 0.0
29788 : 0.0
31220 : 0.0
31358 : 0.0
31669 : 0.0
31877 : 0.0
33887 : 0.0
34000 : 0.0
36187 : 0.0
36782 : 0.0
37515 : 0.0
37700 : 0.0
38317 : 0.0
39151 : 0.0
39363 : 0.0
39481 : 0.0
40163 : 0.0
40468 : 0.0
41216 : 0.0
41692 : 0.0
41835 : 0.0
41922 : 0.0
42328 : 0.0
42617 : 0.0
45000 : 0.0
46926 : 0.0
47643 : 0.0
47668 : 0.0
51876 : 0.0
52122 : 0.0
52321 : 0.0
52426 : 0.0
52853 : 0.0
53812 : 0.0
56490 : 0.0
57247 : 0.0
57629 : 0.0
59687 : 0.0
63435 : 0.0
63646 : 0.0
65933 : 0.0
67042 : 0.0
67406 : 0.0
68480 : 0.0
69726 : 0.0
69734 : 0.0
70248 : 0.0
70272 : 0.0
70984 : 0.0
71659 : 0.0
72084 : 0.0
73366 : 0.0
75350 : 0.0
77527 : 0.0
78295 : 0.0
80640 : 0.0
81563

## Testing Pearson

In [95]:
%%time

feature = 'Energy'
feature_list = []
pearson_list = []
i = 0

for val in df[feature]:   
#     val = np.array([val])
    feature_list.append(similarity_metric(df[feature][45000], val))
    pearson_score = feature_list[i].pearson()
    pearson_list.append(pearson_score)
    i += 1

Wall time: 9.88 s


In [96]:
print(len(pearson_list))
print(pearson_list[25:35])

169909
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [97]:
for i in range(0, len(pearson_list)):
    if pearson_list[i] <= 0:
        print(i,':', pearson_list[i])

## Testing Euclidean

In [98]:
%%time

# The code basis for one target to all values in the same column
 
feature_list = []
euclidean_list = []

feature = 'Tempo'
id_num = 4500 
i = 0 # Index

# lines = iter(open('something.txt', 'r'))
# for val in lines:
#     if val == "!":
#         lines.next()
#         continue
#     <etc>

for val in df[feature]: 
    
    #TODO fix this, need to assign 1 when the number is the same
    feature_list.append(similarity_metric(df[feature][id_num], val))
    #if i == id_num:
    #    val = 1.0
    euclidean_score = feature_list[i].euclidean()
    euclidean_list.append(euclidean_score)
    i += 1 
    
#print(euclidean_list)
#print(tempo_list) #Indent this if you want one-by-one printing
#tempo_list = np.array(tempo_list)

Wall time: 3.86 s


In [99]:
print(len(euclidean_list))
print(euclidean_list[25:35])

169909
[26.33699999999999, 17.11099999999999, 32.152, 18.776999999999973, 13.361000000000004, 20.132000000000005, 2.0490000000000066, 12.146, 72.468, 20.700000000000003]


In [100]:
for i in range(0, len(euclidean_list)):
    if euclidean_list[i] <= 0:
        print(i,':', euclidean_list[i])

4500 : 0.0
13760 : 0.0


In [105]:
new_value = [[25.223, 25],[220.110, 111]]
new_value = similarity_metric(new_value[0],new_value[1]).euclidean()
new_value

213.01864418167722

In [ ]:
def euclidean_similarity(dict_name, id1, id2):
    import math
    
    def feature_select():
        feature_select = ["Accoustiness", "Danceability", "Energy", "Liveness", "Loudness", "Popularity", "Speechiness", "Tempo", "Valence"]
        for number, feature in enumerate(feature_select, start=1):
            print(number, feature) # Present a list of options to the user for feature choice
    
    try:   
        dict_name = dict_name
    
        if id1 == '': # If the positional argument is entered as '', then ask for an int ID
            id1 = int(input("Please insert your first id for music features: "))
        else:
            id1 = id1
        
        if id2 == '': # If the positional argument is entered as '', then ask for an int ID
            id2 = int(input("Please insert your second id for music features: "))
        else:
            id2 = id2
                
        if id1 == id2: # check to make sure the user is not entering the same ID twice
            print("You can't have the same ID, please choose 2 different IDs.")
            euclidean_similarity(dict_name, '','')
        else:
            print("If you are working with defined artist lists, enter 'Artist'")
            feature_select()
            query = input("Which feature do you want to use for comparison? Enter the feature name or enter 'No' to compare all features. ").capitalize().rstrip()

            if query == '' or query == "No".capitalize(): # if query entry is no or left empty, go here
                print("Comparing all respective features using Euclidean.\n")
                # take values from the dictionary from the end,to avoid the string values at the beginning
                feature_list = list(dict_name[id1].values())[-9:] 
                feature_list2 = list(dict_name[id2].values())[-9:]
                key_list = str(list(dict_name[id1].keys())[-9:]).split(',') # take the associated key names
            
                for i in range(0,9): # Loop through the 9 expected features
                    for value in feature_list, feature_list2: # take values over the loop and compare them
                        x = (feature_list[i]) 
                        y = (feature_list2[i]) 
                        distance = math.sqrt((x - y) ** 2 ) #one-dimensional euclidean formula
                    print(key_list[i].strip('[]').strip(' '), round(distance, 3)) # print all feature metrics
            
            else:
                if query == 'Artist' and len(dict_name) == 2: # must be working with chosen artists
                    x = []
                    y = []
                    
                    # To avoid issues with math such as sqrt, create 2 new lists and assign all values to those lists
                    for value in dict_name[id1]:
                        x.append(value)
                    for value in dict_name[id2]:
                        y.append(value)
                    distance = math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)])) # multi-dimensional euclidean
                    return(print("Euclidean distance of {} and {}:".format(id1, id2), round(distance, 3)))
                
                else: # must be working with default dictionary and values
                    # Assign our queried features to our x and y variables
                    x = dict_name[id1][query]
                    y = dict_name[id2][query]
                    distance = math.sqrt((x - y) ** 2 ) #one-dimensional euclidean formula   
                    print("Euclidean Distance of {} for ID {} and ID {} is".format(query, id1, id2), round(distance, 3))
            
    # Error handling for the function is written here
    except KeyError as keyerror:
        print("That feature doesn't exist.", keyerror)
        euclidean_similarity(dict_name,'','')
    except ValueError as valueerror:
        print("Your entry is invalid, please make sure your entry was the correct format.")
        euclidean_similarity(dict_name,'','')
    except TypeError as typeerror:
        print("Invalid type entered.")
    except IndexError:
        print("There was a problem, did you enter your dictionary name correctly?")
    except ZeroDivisionError:
        print("Sorry, but you cannot divide by 0, metric will restart.")
        euclidean_similarity(dict_name,'','')
    except AttributeError:
        print("You can't compare all features of an artist you have defined.")
        euclidean_similarity(dict_name,'','')

## Functions for the Metric choices
### Needs to be adjusted for current use, might be removed

In [ ]:
# Function for metric selection
def metric_choice(dict_name):
    metric = int(input("Which metric would you like to use from the selection: Enter the number: "))
    if metric == 1:
        euclidean_similarity(dict_name, '', '')
    elif metric == 2:
        cosine_similarity(dict_name, '', '')
    elif metric == 3:
        pearson_similarity(dict_name, '', '')
    elif metric == 4:
        jaccard_similarity(dict_name, '', '')
    elif metric == 5:
        manhattan_similarity(dict_name, '', '')
    else:
        print("Your selection is incorrect.")
        if metric == '': # If entry is blank, end the program
            print("You entered nothing, the program will end.")
        else:
            metric_choice(dict_name) # Restart the metric choice
                
def metric_selection():
    metric_select = ["Euclidean", "Cosine", "Pearson", "Jaccard", "Manhattan"]
    for number, metric in enumerate(metric_select, start=1):
        print(number, metric) # Present a list of options to the user for metric choice                